In [81]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [51]:
df1 = pd.read_csv("adult_nb.csv")

In [52]:
df1[:4]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K


In [53]:
df1['income'].value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

In [54]:
df1['income'] = np.where(df1['income'] == ' <=50K', 0, 1)

In [55]:
df1.tail()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,1


In [56]:
df1['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [57]:
for col in df1.columns:
    print(col, ":", sum(df1[col].isna()))


age : 0
workclass : 1836
fnlwgt : 0
education : 0
education_num : 0
marital_status : 0
occupation : 1843
relationship : 0
race : 0
sex : 0
capital_gain : 0
capital_loss : 0
hours_per_week : 0
native_country : 583
income : 0


In [58]:
for value in ["workclass", "occupation", 'native_country']:
    df1[value].replace(np.nan, df1.describe(include = "all")[value][2], inplace = True)

In [59]:
df1.describe(include = 'all')

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
count,32561.000000,32561,3.256100e+04,32561,32561.000000,32561,32561,32561,32561,32561,32561.000000,32561.000000,32561.000000,32561,32561.000000
unique,NaN,8,NaN,16,NaN,7,14,6,5,2,NaN,NaN,NaN,41,NaN
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,NaN
freq,NaN,24532,NaN,10501,NaN,14976,5983,13193,27816,21790,NaN,NaN,NaN,29753,NaN
mean,38.581647,NaN,1.897784e+05,NaN,10.080679,NaN,NaN,NaN,NaN,NaN,1077.648844,87.303830,40.437456,NaN,0.240810
std,13.640433,NaN,1.055500e+05,NaN,2.572720,NaN,NaN,NaN,NaN,NaN,7385.292085,402.960219,12.347429,NaN,0.427581
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,0.000000
25%,28.000000,NaN,1.178270e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,0.000000
50%,37.000000,NaN,1.783560e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,0.000000
75%,48.000000,NaN,2.370510e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,0.000000


In [60]:
df1.describe(include = 'all')['workclass'][2]

' Private'

In [61]:
le_list  = df1.select_dtypes(['object']).columns.tolist()

In [62]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [63]:
for i in le_list:
    df1[i] = le.fit_transform(df1[i])

In [64]:
df1[:4]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,6,77516,9,13,4,0,1,4,1,2174,0,40,38,0
1,50,5,83311,9,13,2,3,0,4,1,0,0,13,38,0
2,38,3,215646,11,9,0,5,1,4,1,0,0,40,38,0
3,53,3,234721,1,7,2,5,0,2,1,0,0,40,38,0


Smote
oversampling 
statified_kfold

In [65]:
scale_cols= df1.columns[:-1].tolist()

In [97]:
df1.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,0.030670,2.624257,-1.063594,-0.335432,1.134721,0.921620,-1.545232,-0.277801,0.393661,0.703061,0.148451,-0.216656,-0.035429,0.261366,0
1,0.837096,1.721073,-1.008692,-0.335432,1.134721,-0.406206,-0.790080,-0.900167,0.393661,0.703061,-0.145918,-0.216656,-2.222119,0.261366,0
2,-0.042641,-0.085295,0.245075,0.181329,-0.420053,-1.734032,-0.286645,-0.277801,0.393661,0.703061,-0.145918,-0.216656,-0.035429,0.261366,0
3,1.057031,-0.085295,0.425795,-2.402474,-1.197440,-0.406206,-0.286645,-0.900167,-1.962591,0.703061,-0.145918,-0.216656,-0.035429,0.261366,0
4,-0.775756,-0.085295,1.408154,-0.335432,1.134721,-0.406206,0.720225,2.211664,-1.962591,-1.422309,-0.145918,-0.216656,-0.035429,-5.352858,0


In [95]:
scale_cols

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country']

In [98]:
scaled_features = {}
for col in scale_cols:
    mean, std = df1[col].mean(), df1[col].std()
    scaled_features[col] = [mean,std]
    df1.loc[:, col] = (df1[col] - mean/std)

In [ ]:
scaled_features = {}

for col in scale_cols:
    mean, std = adult_df[col].mean(), adult_df[col].std()
    scaled_features[col] = [mean, std]
    adult_df.loc[:, col] = (adult_df[col] - mean)/std

In [108]:
features = df1.values[:, :14]
target = df1.values[:, 14]

In [109]:
features[0]

array([ 0.03067009,  2.62425734, -1.06359441, -0.33543178,  1.13472134,
        0.92161979, -1.54523205, -0.27780077,  0.39366148,  0.70306055,
        0.14845062, -0.2166562 , -0.0354289 ,  0.26136598])

In [100]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.33, random_state = 10)

In [101]:
#Model

In [102]:
clf= GaussianNB()
clf.fit(features_train, target_train)
target_pred = clf.predict(features_test)

In [103]:
accuracy_score(target_test,target_pred)

0.8014144798064397

In [104]:
print(confusion_matrix(target_test, target_pred))

[[7721  435]
 [1699  891]]


In [105]:
print(classification_report(target_test,target_pred))

              precision    recall  f1-score   support

         0.0       0.82      0.95      0.88      8156
         1.0       0.67      0.34      0.46      2590

    accuracy                           0.80     10746
   macro avg       0.75      0.65      0.67     10746
weighted avg       0.78      0.80      0.78     10746

